# Ch20: 例外處理機制 | Exception Handling

## Part I: 理論基礎

### 章節概述

**學習時數**: 2 小時  
**難度等級**: ⭐⭐⭐ (中級)  
**先備知識**: 函式、檔案處理基礎

**本章重點**:
1. 理解例外處理的必要性
2. 掌握 try-except-else-finally 完整機制
3. 學會處理常見的執行時期錯誤
4. 設計穩健的錯誤處理架構

---

## 什麼是例外？

**例外 (Exception)** 是程式執行時發生的錯誤事件，會中斷正常的程式流程。

**兩種錯誤類型**:

1. **語法錯誤 (Syntax Error)**: 程式碼寫錯，無法執行
```python
# 語法錯誤：Python 直接拒絕執行
if True  # 缺少冒號
    print("Hello")
```

2. **例外 (Exception)**: 語法正確，但執行時出錯
```python
# 例外：語法正確，但執行時會出錯
result = 10 / 0  # ZeroDivisionError: 除以零
```

**為何需要例外處理？**
- 使用者輸入無法預測 (輸入文字而非數字)
- 外部資源可能不存在 (檔案、網路)
- 系統資源可能不足 (記憶體、磁碟空間)
- 讓程式更穩健 (不會隨便崩潰)

---

## 常見的內建例外

| 例外類型 | 觸發時機 | 範例 |
|:--------|:---------|:-----|
| `ValueError` | 值的型別正確但內容不合理 | `int("abc")` |
| `TypeError` | 操作或函式套用於不適當的型別 | `"3" + 3` |
| `ZeroDivisionError` | 除以零 | `10 / 0` |
| `IndexError` | 索引超出範圍 | `[1, 2][5]` |
| `KeyError` | 字典中找不到鍵 | `{"a": 1}["b"]` |
| `FileNotFoundError` | 檔案不存在 | `open("no_file.txt")` |
| `AttributeError` | 物件沒有該屬性 | `"abc".non_exist()` |
| `NameError` | 變數名稱不存在 | `print(undefined_var)` |

---

## Part II: 實作演練

### 範例 1: try-except 基礎 (檔案讀取)

**問題**: 如果檔案不存在，程式會崩潰。如何優雅地處理？

In [ ]:
# ❌ 沒有例外處理：檔案不存在時程式會崩潰
def read_file_bad(filename):
    file = open(filename, 'r', encoding='utf-8')
    content = file.read()
    file.close()
    return content

# 測試：這會崩潰
# content = read_file_bad("non_existent.txt")  # FileNotFoundError!

In [ ]:
# ✅ 使用 try-except 處理例外
def read_file_good(filename):
    try:
        # 嘗試執行可能出錯的程式碼
        file = open(filename, 'r', encoding='utf-8')
        content = file.read()
        file.close()
        return content
    except FileNotFoundError:
        # 如果發生 FileNotFoundError，執行這裡
        print(f"錯誤：檔案 '{filename}' 不存在")
        return None

# 測試
content = read_file_good("non_existent.txt")
print(f"讀取結果: {content}")

In [ ]:
# 📖 解析：try-except 的結構

# try:
#     # 可能出錯的程式碼
#     risky_operation()
# except SpecificException:
#     # 處理特定例外
#     handle_error()

# 執行流程：
# 1. 執行 try 區塊
# 2. 如果沒有例外，跳過 except 區塊
# 3. 如果發生 SpecificException，執行 except 區塊
# 4. 如果發生其他例外，程式仍會崩潰

print("✅ 範例 1 完成：學會基本的 try-except 結構")

### 範例 2: 多重 except 子句 (不同錯誤處理)

**問題**: 使用者輸入可能有多種錯誤，如何分別處理？

In [ ]:
def safe_divide():
    """安全的除法運算，處理多種可能的錯誤"""
    try:
        numerator = int(input("請輸入被除數: "))
        denominator = int(input("請輸入除數: "))
        result = numerator / denominator
        print(f"結果: {result}")
        return result
    
    except ValueError:
        # 處理輸入不是數字的情況
        print("錯誤：請輸入有效的數字！")
        return None
    
    except ZeroDivisionError:
        # 處理除以零的情況
        print("錯誤：除數不能為零！")
        return None

# 測試（在 Jupyter 中手動執行）
# safe_divide()
# 嘗試輸入：
# 1. 正常數字：10, 2 → 成功
# 2. 非數字：abc, 5 → ValueError
# 3. 除以零：10, 0 → ZeroDivisionError

In [ ]:
# 進階：取得例外的詳細資訊
def safe_divide_v2():
    try:
        numerator = int(input("請輸入被除數: "))
        denominator = int(input("請輸入除數: "))
        result = numerator / denominator
        print(f"結果: {result}")
        return result
    
    except ValueError as e:
        # 使用 as e 取得例外物件
        print(f"數值錯誤: {e}")
        return None
    
    except ZeroDivisionError as e:
        print(f"除法錯誤: {e}")
        return None
    
    except Exception as e:
        # 捕捉所有其他例外（保險措施）
        print(f"未預期的錯誤: {type(e).__name__} - {e}")
        return None

# safe_divide_v2()
print("✅ 範例 2 完成：學會多重 except 與取得例外資訊")

### 範例 3: else 子句的使用 (成功時執行)

**問題**: 如何清楚區分「成功時的處理」與「錯誤處理」？

**else 子句**: 只在 try 區塊沒有發生例外時執行

In [ ]:
def read_and_process_file(filename):
    """讀取檔案並處理內容"""
    try:
        file = open(filename, 'r', encoding='utf-8')
    except FileNotFoundError:
        print(f"錯誤：檔案 '{filename}' 不存在")
    else:
        # 只有在檔案成功開啟時才執行
        print(f"✅ 成功開啟檔案 '{filename}'")
        content = file.read()
        print(f"檔案內容長度: {len(content)} 字元")
        file.close()
        print("檔案已關閉")
        return content

# 測試：讀取不存在的檔案
read_and_process_file("test.txt")

In [ ]:
# 建立測試檔案並再次測試
# 先建立一個測試檔案
with open("test.txt", "w", encoding="utf-8") as f:
    f.write("這是測試內容\n第二行")

# 現在讀取應該成功
content = read_and_process_file("test.txt")
print(f"\n返回值: {content}")

In [ ]:
# 📖 解析：為何使用 else？

# ❌ 不清楚的寫法：成功處理寫在 try 裡面
def unclear_version(filename):
    try:
        file = open(filename, 'r')
        content = file.read()  # 如果這裡出錯，不知道是開檔還是讀檔的問題
        process(content)       # 如果這裡出錯，也會被 except 捕捉
        file.close()
    except FileNotFoundError:
        print("檔案不存在")

# ✅ 清楚的寫法：用 else 明確分離
def clear_version(filename):
    try:
        file = open(filename, 'r')  # 只有開檔操作在 try 中
    except FileNotFoundError:
        print("檔案不存在")
    else:
        # 成功開啟後的處理
        content = file.read()
        # process(content)  # 這裡的錯誤不會被上面的 except 捕捉
        file.close()

print("✅ 範例 3 完成：學會使用 else 子句分離成功與失敗邏輯")

### 範例 4: finally 子句 (清理資源)

**問題**: 如何確保資源一定被釋放？(即使發生例外)

**finally 子句**: 無論是否發生例外，都一定會執行

In [ ]:
def process_file_with_finally(filename):
    """使用 finally 確保檔案一定會被關閉"""
    file = None
    try:
        print("1. 嘗試開啟檔案...")
        file = open(filename, 'r', encoding='utf-8')
        
        print("2. 讀取檔案內容...")
        content = file.read()
        
        print("3. 處理內容...")
        # 模擬處理過程中發生錯誤
        # raise ValueError("處理內容時發生錯誤")
        
        return content
    
    except FileNotFoundError:
        print("❌ 例外：檔案不存在")
        return None
    
    except ValueError as e:
        print(f"❌ 例外：{e}")
        return None
    
    finally:
        # 無論如何都會執行
        print("4. finally: 清理資源...")
        if file is not None:
            file.close()
            print("   檔案已關閉")
        print("5. finally 執行完畢")

# 測試 1: 正常情況
print("=== 測試 1: 檔案存在 ===")
result = process_file_with_finally("test.txt")
print(f"返回值: {result}\n")

In [ ]:
# 測試 2: 檔案不存在
print("=== 測試 2: 檔案不存在 ===")
result = process_file_with_finally("non_existent.txt")
print(f"返回值: {result}")

In [ ]:
# 📖 finally 的執行時機演示
def finally_timing_demo():
    try:
        print("A: try 開始")
        return "從 try 返回"
        print("B: try 結束（不會執行）")
    finally:
        print("C: finally 執行（在 return 之前！）")

result = finally_timing_demo()
print(f"D: 函式外，返回值 = {result}")

# 輸出順序：
# A: try 開始
# C: finally 執行（在 return 之前！）
# D: 函式外，返回值 = 從 try 返回

print("\n✅ 範例 4 完成：學會使用 finally 清理資源")

### 範例 5: 例外階層與捕捉順序 (父子類例外)

**問題**: 為何例外的捕捉順序很重要？

**原理**: Python 例外遵循繼承階層，父類例外可以捕捉所有子類例外

In [ ]:
# 查看例外的繼承關係
print("例外繼承階層示範：")
print(f"FileNotFoundError 的父類: {FileNotFoundError.__bases__}")
print(f"OSError 的父類: {OSError.__bases__}")
print(f"Exception 的父類: {Exception.__bases__}")

# 階層關係：
# BaseException
#   └── Exception
#         └── OSError
#               └── FileNotFoundError

print("\n檢查繼承關係：")
print(f"FileNotFoundError 是 OSError 的子類嗎？ {issubclass(FileNotFoundError, OSError)}")
print(f"FileNotFoundError 是 Exception 的子類嗎？ {issubclass(FileNotFoundError, Exception)}")

In [ ]:
# ❌ 錯誤：父類在前，子類永遠捕捉不到
def wrong_order():
    try:
        file = open("non_existent.txt", 'r')
    except OSError:  # 父類
        print("被 OSError 捕捉")
    except FileNotFoundError:  # 子類（永遠執行不到）
        print("被 FileNotFoundError 捕捉")

# Python 3.11+ 會警告：except FileNotFoundError 永遠不會執行
wrong_order()

In [ ]:
# ✅ 正確：具體例外在前，一般例外在後
def correct_order():
    try:
        file = open("non_existent.txt", 'r')
    except FileNotFoundError:  # 子類（具體）
        print("被 FileNotFoundError 捕捉（具體處理）")
    except OSError:  # 父類（一般）
        print("被 OSError 捕捉（一般處理）")
    except Exception:  # 更父類（保險）
        print("被 Exception 捕捉（最後保險）")

correct_order()

In [ ]:
# 實務範例：處理檔案操作的多種錯誤
def robust_file_operation(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            content = f.read()
        return content
    
    except FileNotFoundError:
        # 最具體：檔案不存在
        print(f"錯誤：找不到檔案 '{filename}'")
        return None
    
    except PermissionError:
        # 具體：沒有權限
        print(f"錯誤：沒有權限讀取 '{filename}'")
        return None
    
    except OSError as e:
        # 一般：其他作業系統相關錯誤
        print(f"系統錯誤：{e}")
        return None
    
    except Exception as e:
        # 最一般：未預期的錯誤
        print(f"未知錯誤：{type(e).__name__} - {e}")
        return None

# 測試
robust_file_operation("non_existent.txt")

print("\n✅ 範例 5 完成：學會正確的例外捕捉順序（具體→一般）")

### 範例 6: 取得例外詳細資訊 (traceback)

**問題**: 如何取得完整的錯誤堆疊資訊，方便除錯？

In [ ]:
import traceback
import sys

def level_3():
    """第三層函式：發生錯誤"""
    numbers = [1, 2, 3]
    return numbers[10]  # IndexError!

def level_2():
    """第二層函式：呼叫 level_3"""
    return level_3()

def level_1():
    """第一層函式：呼叫 level_2"""
    return level_2()

# 方法 1: 基本例外資訊
try:
    level_1()
except IndexError as e:
    print("=== 方法 1: 基本例外資訊 ===")
    print(f"例外類型: {type(e).__name__}")
    print(f"例外訊息: {e}")
    print(f"例外參數: {e.args}")

In [ ]:
# 方法 2: 使用 traceback 模組取得完整堆疊
try:
    level_1()
except IndexError as e:
    print("\n=== 方法 2: 完整 Traceback ===")
    print("完整錯誤堆疊：")
    traceback.print_exc()
    
    print("\n取得 traceback 字串（可記錄到日誌）：")
    tb_str = traceback.format_exc()
    print(tb_str)

In [ ]:
# 方法 3: 取得例外的詳細資訊（進階）
try:
    level_1()
except IndexError as e:
    print("\n=== 方法 3: 詳細除錯資訊 ===")
    
    # 取得例外資訊
    exc_type, exc_value, exc_traceback = sys.exc_info()
    
    print(f"例外類型: {exc_type.__name__}")
    print(f"例外值: {exc_value}")
    
    print("\n呼叫堆疊：")
    for frame_summary in traceback.extract_tb(exc_traceback):
        print(f"  檔案: {frame_summary.filename}")
        print(f"  函式: {frame_summary.name}")
        print(f"  行號: {frame_summary.lineno}")
        print(f"  程式碼: {frame_summary.line}")
        print()

In [ ]:
# 實務應用：記錄錯誤到日誌
import logging

# 設定日誌
logging.basicConfig(
    level=logging.ERROR,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def risky_operation(x):
    """可能出錯的操作"""
    try:
        result = 10 / x
        return result
    except ZeroDivisionError:
        # logging.exception() 會自動記錄 traceback
        logging.exception("除以零錯誤發生")
        return None
    except Exception:
        logging.exception("未預期的錯誤")
        return None

# 測試
print("\n=== 測試日誌記錄 ===")
result = risky_operation(0)
print(f"返回值: {result}")

print("\n✅ 範例 6 完成：學會取得並記錄詳細的例外資訊")

---

## Part III: 本章總結

### 知識回顧

**1. try-except-else-finally 完整結構**
```python
try:
    # 可能出錯的程式碼
    risky_code()
except SpecificError:
    # 處理特定錯誤
    handle_error()
else:
    # 沒有錯誤時執行
    success_code()
finally:
    # 無論如何都執行（清理資源）
    cleanup()
```

**2. 執行流程**
- **無例外**: try → else → finally
- **有例外且被捕捉**: try → except → finally
- **有例外但未捕捉**: try → finally → 程式崩潰

**3. 例外捕捉順序原則**
- ✅ 具體例外在前，一般例外在後
- ❌ 不要使用裸 `except:` (會捕捉所有例外，包括 KeyboardInterrupt)
- ❌ 不要吞噬例外 (捕捉後不處理)

**4. 取得例外資訊**
- `except ExceptionType as e`: 取得例外物件
- `str(e)`: 取得錯誤訊息
- `type(e).__name__`: 取得例外類型名稱
- `traceback.format_exc()`: 取得完整堆疊字串

---

### 常見誤區

| 錯誤做法 | 為何錯誤 | 正確做法 |
|:--------|:---------|:---------|
| `except:` | 會捕捉所有例外（含 Ctrl+C） | `except Exception:` |
| 父類例外在前 | 子類永遠捕捉不到 | 具體例外在前 |
| 捕捉後不處理 | 隱藏錯誤，難以除錯 | 至少記錄日誌 |
| 所有程式碼都包在 try | 無法確定錯誤來源 | 只包可能出錯的部分 |
| 忘記清理資源 | 資源洩漏 | 使用 finally 或 with |

---

### 自我檢核

完成本講義後，您應該能夠：

**基礎能力**
- [ ] 使用 try-except 處理基本例外
- [ ] 使用 `except ExceptionType as e` 取得例外物件
- [ ] 使用 finally 清理資源
- [ ] 知道常見例外類型的觸發條件

**進階能力**
- [ ] 撰寫多重 except 處理不同錯誤
- [ ] 使用 else 分離成功與失敗邏輯
- [ ] 正確排列例外捕捉順序
- [ ] 使用 traceback 模組取得詳細資訊

**應用能力**
- [ ] 設計完整的例外處理架構
- [ ] 判斷何時該捕捉、何時該傳播例外
- [ ] 結合 logging 模組記錄錯誤

---

### 延伸閱讀

**Python 官方文件**
- [Errors and Exceptions](https://docs.python.org/3/tutorial/errors.html)
- [Built-in Exceptions](https://docs.python.org/3/library/exceptions.html)

**下一步學習**
- **Ch21: 自訂例外與 raise** - 學習主動拋出例外、設計自己的例外類別
- **Ch22: 除錯技術** - 掌握 logging、pdb 除錯器
- **02-worked-examples.ipynb** - 4個實務案例詳解
- **03-practice.ipynb** - 8題即時練習

---

### 課後練習預告

**03-practice.ipynb** (課堂練習):
1. 安全的使用者輸入驗證
2. 檔案讀寫錯誤處理
3. 字典操作例外處理
4. 列表索引錯誤處理
5. 多層函式呼叫的例外傳播
6. 資源管理與清理
7. 例外資訊記錄
8. 綜合應用：銀行系統錯誤處理

**04-exercises.ipynb** (課後作業):
- 12題，涵蓋基礎、中級、進階、挑戰四個等級
- 包含實務場景：網路爬蟲、資料處理、API 呼叫

---

**恭喜完成 Ch20 例外處理機制的學習！**

現在您已經掌握了 Python 穩健程式設計的核心技能。

記住：好的例外處理不是到處加 try-except，而是在適當的地方處理預期的錯誤，讓未預期的錯誤能夠明確地暴露出來。